# Лабораторная работа №2

### Выполнили: 
* Иванов Александр Константинович, 368220 
* Братушка Никита Игоревич, 367924
* Проворов Николай Дмитриевич, 368690

# Импорт библиотек

In [63]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.metrics import classification_report, r2_score, mean_absolute_error, mean_squared_error

# Разбиение данных

Считываем данные.

In [4]:
sales = pd.read_csv('Advertising_modified.csv')
heartrates = pd.read_csv('heart_modified.csv')

Далее выполним разбиение выборок на обучающающие и тестовые датасеты в соотношении (80/20).

In [33]:
# features and goal for sales
x1 = sales.iloc[:, :-1]
y1 = sales['sales']
# features and goal for heart
x2 = heartrates.iloc[:,:-1]
y2 = heartrates['target']

#split the data
xs_train, xs_test, ys_train, ys_test = train_test_split(x1, y1, random_state=25, test_size=0.2)
# xs_train.shape, xs_test.shape, ys_train.shape, ys_test.shape
xh_train, xh_test, yh_train, yh_test = train_test_split(x2, y2, random_state=27, test_size=0.2)
# xh_train.shape, xh_test.shape, yh_train.shape, yh_test.shape

((242, 13), (61, 13), (242,), (61,))

# Классификация

В этом задании мы должны рассмотреть стохастический градиентный спуск (SGDClassifier) со следующими функциями потерь:

* hinge - модель SVM
* squared_hinge - модель SVM
* perceptron - модель перцептрона (подробнее - [wiki](https://en.wikipedia.org/wiki/Perceptron))

## Обучение

Для определения наилучшей архитектуры мы будем использовать GridSearchCV.

In [29]:
# параметры SGDClassifier, на основании будут рассматриваться различные архитектуры 
grid = {
    'loss': ['hinge', 'squared_hinge', 'perceptron'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01, 0.05],
    'max_iter':[1000],
}
# Использоваем GridSearch для определения самой эффективной архитектуры
models_classifier = GridSearchCV(
    estimator=SGDClassifier(),
    param_grid=grid,
    n_jobs=-1,
    scoring= ['accuracy', 'precision'],
    refit = 'accuracy'
)
models_classifier.fit(xh_train, yh_train)
best_classifier = models_classifier.best_estimator_
# выведем наилучшие параметры, которые были получены
print('Самая эффективная архитектура:')
print(models_classifier.best_params_)
print(f'Accuracy: {models_classifier.best_score_}')

/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteratio

Самая эффективная архитектура:
{'alpha': 0.01, 'loss': 'hinge', 'max_iter': 1000, 'penalty': 'l2'}
Accuracy: 0.8384353741496599


/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Результаты

Оценим результаты самой эффективной модели на тестовых данных

In [32]:
yh_pred = best_classifier.predict(xh_test)
results_h = classification_report(yh_test, yh_pred)
print(results_h)

              precision    recall  f1-score   support

           0       0.92      0.85      0.88        27
           1       0.89      0.94      0.91        34

    accuracy                           0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61



Можно отметить, мы получили высокую точность (accuracy) - 90%. Также стоит отметить, что у значение recall заметно выше для 1 - наличия заболевания. Точность при этом чуть ниже, нежели при определении 0 - отсутсвия заболевания. Однако лучше, с нашей точки зрения, иметь ложно положительные (с заболеванием) случае, чем ложно отрицательные в случае с сердечными заболеваниями. 

## Использование *minibatch_optimization*

При использовании тохастического спуска трек обучения может быть запутанным и не всегда приводить к минимуму функции потерь (например, при недостаточном количестве эпох).

Поэтому мы попробуем использовать минивыборки. Всего у нас 242 записи в обучающей выборке. Поэтому выберем размер минивыборки, равным `22`.

In [46]:
batch_size = 22
n_batches = xh_train.shape[0]/batch_size # 11
# выставляем лучшую архитектуру, полученную в предыдущем пункте
partial_model = SGDClassifier(loss='hinge', alpha=0.01, penalty='l2')
# разбиваем данные на минибатчи
xh_batches = np.array_split(xh_train, n_batches)
yh_batches = np.array_split(yh_train, n_batches)
# обучаем
for i, (x_batch, y_batch) in enumerate(zip(xh_batches, yh_batches)):
    if i == 0:
        partial_model.partial_fit(x_batch, y_batch, classes=[0, 1])
    else:
        partial_model.partial_fit(x_batch, y_batch)

yh_part_pred = partial_model.predict(xh_test)
results_part_h = classification_report(yh_test, yh_part_pred)
print(results_part_h)

              precision    recall  f1-score   support

           0       0.95      0.70      0.81        27
           1       0.80      0.97      0.88        34

    accuracy                           0.85        61
   macro avg       0.88      0.84      0.84        61
weighted avg       0.87      0.85      0.85        61



/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


# Регрессия

В качестве фукнции ошибки для SGDRegressor мы будем использовать:
* squared_error
* huber
* epsilon_insensitive

## Обучение

In [58]:
# параметры SGDRegressor, на основании будут рассматриваться различные архитектуры 
grid = {
    'loss': ['squared_error', 'huber', 'epsilon_insensitive'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01, 0.05],
    'max_iter':[1000],
}
# Использоваем GridSearch для определения самой эффективной архитектуры
models_regressor = GridSearchCV(
    estimator=SGDRegressor(),
    param_grid=grid,
    n_jobs=-1,
    scoring= ['neg_mean_squared_error', 'neg_mean_absolute_error'],
    refit = 'neg_mean_absolute_error'
)
models_regressor.fit(xs_train, ys_train)
best_regressor = models_regressor.best_estimator_
# выведем наилучшие параметры, которые были получены
results_s_test = pd.DataFrame(models_regressor.cv_results_)

print('Таблица с результатами сравнениня моделей:')
results_s_test = results_s_test[["param_loss", "param_penalty", "param_alpha", "mean_test_neg_mean_absolute_error", "mean_test_neg_mean_squared_error", "rank_test_neg_mean_absolute_error", "rank_test_neg_mean_squared_error"]]
results_s_test.columns = ["loss", "penalty", "alpha", "MAE", "MSE", "ranking (MAE)", "ranking (MSE)"]
results_s_test["MAE"], results_s_test["MSE"] = [-results_s_test["MAE"], -results_s_test["MSE"]]
display(results_s_test)
print('Самая эффективная архитектура:')
print(models_regressor.best_params_)
print(f'MAE: {-models_regressor.best_score_}')

Таблица с результатами сравнениня моделей:


,loss,penalty,alpha,MAE,MSE,ranking (MAE),ranking (MSE)
0,squared_error,l1,0.0001,0.262290,0.124371,20,7
1,squared_error,l2,0.0001,0.262036,0.124122,15,3
2,squared_error,elasticnet,0.0001,0.262310,0.124393,21,8
3,huber,l1,0.0001,0.402217,0.326326,27,27
4,huber,l2,0.0001,0.402200,0.326083,26,26
5,huber,elasticnet,0.0001,0.401876,0.325824,25,25
6,epsilon_insensitive,l1,0.0001,0.250212,0.130448,4,15
7,epsilon_insensitive,l2,0.0001,0.250024,0.130783,2,16
8,epsilon_insensitive,elasticnet,0.0001,0.251499,0.131304,12,20
9,squared_error,l1,0.0010,0.262087,0.124196,16,4


Самая эффективная архитектура:
{'alpha': 0.05, 'loss': 'epsilon_insensitive', 'max_iter': 1000, 'penalty': 'l1'}
MAE: 0.24650405170818926


## Результаты

In [67]:
ys_pred = best_regressor.predict(xs_test)
results_s = {'R2-score':r2_score(ys_test, ys_pred),
             'MAE':mean_absolute_error(ys_test, ys_pred),
             'MSE':mean_squared_error(ys_test, ys_pred)}
display(results_s)

{'R2-score': 0.9150809762819287,
 'MAE': 0.23192228128407053,
 'MSE': 0.09437382408058986}

## Использование *minibatch_optimization*

In [98]:
batch_size = 99
n_batches = xs_train.shape[0]/batch_size
# выставляем лучшую архитектуру, полученную в предыдущем пункте
partial_model = SGDRegressor(loss='epsilon_insensitive', alpha=0.05, penalty='l1')
# разбиваем данные на минибатчи
xs_batches = np.array_split(xs_train, n_batches)
ys_batches = np.array_split(ys_train, n_batches)
# обучаем
for i, (x_batch, y_batch) in enumerate(zip(xs_batches, ys_batches)):
        partial_model.partial_fit(x_batch, y_batch)

ys_part_pred = partial_model.predict(xs_test)
results_part_s = {'R2-score':r2_score(ys_test, ys_part_pred),
             'MAE':mean_absolute_error(ys_test, ys_part_pred),
             'MSE':mean_squared_error(ys_test, ys_part_pred)}
print(results_part_s)

{'R2-score': 0.6746100805342773, 'MAE': 0.44441628871447153, 'MSE': 0.36161851223356134}


/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/nikitabratushka/Vs_code/MachineLearning/.venv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
